In [1]:
pip install datasets

In [2]:
pip install chardet

In [3]:
# Import necessary libraries
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
from datasets import load_dataset, DatasetDict
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import pandas as pd
import os

In [4]:
# Load tokenizer and pre-trained model
tokenizer = AutoTokenizer.from_pretrained("lakshyakh93/deberta_finetuned_pii")
model = AutoModelForTokenClassification.from_pretrained("lakshyakh93/deberta_finetuned_pii")

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
# # Load the mini dataset (200 row)
# import chardet

# filepath = os.path.join(os.getcwd(), 'mini_df_200_row_biolabels.csv')

# # Detect encoding
# with open(filepath, 'rb') as f:
#     result = chardet.detect(f.read())
#     print("Detected file encoding:", result['encoding'])

# # Read the file using the detected encoding
# mini_df = pd.read_csv(filepath, encoding=result['encoding'])

# # Proportional split ratios
# train_ratio = 0.7
# val_ratio = 0.15
# test_ratio = 0.15

# # Since you have 200 rows, no need for the 3,000 cap, as this won't be needed
# train_size = int(len(mini_df) * train_ratio)
# val_size = int(len(mini_df) * val_ratio)
# test_size = len(mini_df) - train_size - val_size  # Remaining rows for the test set

# # Split the dataset into train, validation, and test sets
# train_df, temp_df = train_test_split(mini_df, test_size=1 - train_ratio, random_state=42)
# val_df, test_df = train_test_split(temp_df, test_size=test_ratio / (test_ratio + val_ratio), random_state=42)

# # Print dataset sizes to confirm
# print(f"Training size: {len(train_df)}")
# print(f"Validation size: {len(val_df)}")
# print(f"Test size: {len(test_df)}")

In [6]:
# # For the cleaned mini dataframe
# # Create a mapping for the bio_labels
# data_label_list = []

# # Extract unique labels from the 'bio_labels' column
# def get_unique_labels(df):
#     unique_labels = set()
#     for labels in df['bio_labels']:  # Iterate through the 'bio_labels' column
#         # Since bio_labels is a string representation of a list, we need to convert it back to a list
#         labels = eval(labels)  # Convert the string to a list
#         unique_labels.update(labels)  # Add all labels in the current row to the set
#     return sorted(unique_labels)  # Return sorted labels for consistency

# # Apply the function to the entire DataFrame
# unique_labels = get_unique_labels(mini_df)  # Run on the whole DataFrame

# # Print the unique labels
# print("Data's Unique BIO Labels:", unique_labels)

# # Combine with the existing label_list (if applicable)
# data_label_list = sorted(set(data_label_list + unique_labels))  # Ensure no duplicates
# print("Data's Final Label List:", data_label_list)
# print(len(data_label_list))  # Number of unique labels

# data_label_to_id = {label: idx for idx, label in enumerate(data_label_list)}
# data_id_to_label = {idx: label for label, idx in data_label_to_id.items()}

In [7]:
# Calculate the proportional sizes
total_samples = 2000
train_ratio = 7
val_ratio = 1.5
test_ratio = 1.5

total_ratio = train_ratio + val_ratio + test_ratio

train_size = int(total_samples * (train_ratio / total_ratio))
val_size = int(total_samples * (val_ratio / total_ratio))
test_size = total_samples - train_size - val_size

# Load dataset
dataset = load_dataset("Isotonic/pii-masking-200k")

# First, split off the test and validation sets
first_split = dataset["train"].train_test_split(test_size=(val_size + test_size)/len(dataset["train"]), seed=42)

# Then split the remaining data into train
train_split = first_split["train"].shuffle(seed=42).select(range(train_size))
val_test_split = first_split["test"].shuffle(seed=42).select(range(val_size + test_size))
val_split = val_test_split.select(range(val_size))
test_split = val_test_split.select(range(val_size, val_size + test_size))

# Create the final DatasetDict
dataset_splits = DatasetDict({
    "train": train_split,
    "validation": val_split,
    "test": test_split
})

print(f"Training size: {len(dataset_splits['train'])}")
print(f"Validation size: {len(dataset_splits['validation'])}")
print(f"Test size: {len(dataset_splits['test'])}")

Training size: 1400
Validation size: 300
Test size: 300


In [8]:
# Convert test_dataset into a dataframe
pii200k_train = dataset_splits['train'].to_pandas()
pii200k_test = dataset_splits['test'].to_pandas()
eng_train = pii200k_train[pii200k_train['language'] == 'en']
eng_test = pii200k_test[pii200k_test['language'] == 'en']
print(len(eng_train))
print(len(eng_test))

267
68


In [9]:
# Filter rows where 'bio_labels' is not null (though unlikely given your data)
valid_eng_train = eng_train[eng_train['bio_labels'].notnull()]
valid_eng_test = eng_test[eng_test['bio_labels'].notnull()]

import numpy as np
import re

def clean_bio_labels(bio_labels):
    # Flatten the numpy array of bio_labels and remove any non-printable characters
    bio_labels = [label for label in bio_labels.flatten() if isinstance(label, str)]
    cleaned_labels = [re.sub(r'[^\x20-\x7E]', '', label) for label in bio_labels]
    return cleaned_labels

# Apply the cleaning function to each bio_labels entry
eng_train['bio_labels'] = eng_train['bio_labels'].apply(clean_bio_labels)
eng_test['bio_labels'] = eng_test['bio_labels'].apply(clean_bio_labels)

# Check the resulting DataFrame
print(f"Number of valid rows in eng_train: {len(eng_train)}")
print(f"Number of valid rows in eng_test: {len(eng_test)}")

Number of valid rows in eng_train: 267
Number of valid rows in eng_test: 68


<ipython-input-9-24ddafc033f7>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eng_train['bio_labels'] = eng_train['bio_labels'].apply(clean_bio_labels)
<ipython-input-9-24ddafc033f7>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eng_test['bio_labels'] = eng_test['bio_labels'].apply(clean_bio_labels)


In [10]:
data_label_list = []

# Extract unique BIO labels
unique_labels = set()

# Iterate through the 'bio_labels' column
for labels in eng_train['bio_labels']:
    unique_labels.update(labels)  # Add all unique labels from each array

# Convert to sorted list for consistency
unique_labels = sorted(unique_labels)

print("Data's Unique BIO Labels:", unique_labels)
print(f"Number of unique labels: {len(unique_labels)}")

# Combine with the existing label_list (if applicable)
data_label_list = sorted(set(data_label_list + unique_labels))  # Ensure no duplicates
print("Data's Final Label List:", data_label_list)
print(len(data_label_list))  # Number of unique labels

data_label_to_id = {label: idx for idx, label in enumerate(data_label_list)}
data_id_to_label = {idx: label for label, idx in data_label_to_id.items()}

Data's Unique BIO Labels: ['B-ACCOUNTNAME', 'B-ACCOUNTNUMBER', 'B-AGE', 'B-AMOUNT', 'B-BIC', 'B-BITCOINADDRESS', 'B-BUILDINGNUMBER', 'B-CITY', 'B-COMPANYNAME', 'B-COUNTY', 'B-CREDITCARDCVV', 'B-CREDITCARDISSUER', 'B-CREDITCARDNUMBER', 'B-CURRENCY', 'B-CURRENCYCODE', 'B-CURRENCYNAME', 'B-CURRENCYSYMBOL', 'B-DATE', 'B-DOB', 'B-EMAIL', 'B-ETHEREUMADDRESS', 'B-EYECOLOR', 'B-FIRSTNAME', 'B-GENDER', 'B-HEIGHT', 'B-IBAN', 'B-IP', 'B-IPV4', 'B-IPV6', 'B-JOBAREA', 'B-JOBTITLE', 'B-JOBTYPE', 'B-LASTNAME', 'B-LITECOINADDRESS', 'B-MAC', 'B-MASKEDNUMBER', 'B-MIDDLENAME', 'B-NEARBYGPSCOORDINATE', 'B-ORDINALDIRECTION', 'B-PASSWORD', 'B-PHONEIMEI', 'B-PHONENUMBER', 'B-PIN', 'B-PREFIX', 'B-SECONDARYADDRESS', 'B-SEX', 'B-SSN', 'B-STATE', 'B-STREET', 'B-TIME', 'B-URL', 'B-USERAGENT', 'B-USERNAME', 'B-VEHICLEVIN', 'B-VEHICLEVRM', 'B-ZIPCODE', 'I-ACCOUNTNAME', 'I-ACCOUNTNUMBER', 'I-AGE', 'I-AMOUNT', 'I-BIC', 'I-BITCOINADDRESS', 'I-BUILDINGNUMBER', 'I-CITY', 'I-COMPANYNAME', 'I-COUNTY', 'I-CREDITCARDISSUER'

In [11]:
model_label_list = model.config.id2label.values()  # Ensure this matches the fine-tuned model
model_id_to_label = {int(k): v for k, v in model.config.id2label.items()}
model_label_to_id = {v: int(k) for k, v in model_id_to_label.items()}

print("Model's Label to ID:", model_label_to_id)
print("Model's ID to Label:", model_id_to_label)
print(len(model_label_list))

Model's Label to ID: {'B-PREFIX': 0, 'I-PREFIX': 1, 'B-FIRSTNAME': 2, 'I-FIRSTNAME': 3, 'B-MIDDLENAME': 4, 'B-LASTNAME': 5, 'I-LASTNAME': 6, 'O': 7, 'B-JOBDESCRIPTOR': 8, 'B-JOBTITLE': 9, 'I-JOBTITLE': 10, 'B-COMPANY_NAME': 11, 'I-COMPANY_NAME': 12, 'B-JOBAREA': 13, 'B-EMAIL': 14, 'I-EMAIL': 15, 'B-TIME': 16, 'I-TIME': 17, 'B-DATE': 18, 'I-DATE': 19, 'B-URL': 20, 'I-URL': 21, 'B-BITCOINADDRESS': 22, 'I-BITCOINADDRESS': 23, 'B-ETHEREUMADDRESS': 24, 'I-ETHEREUMADDRESS': 25, 'B-ACCOUNTNAME': 26, 'I-ACCOUNTNAME': 27, 'B-IBAN': 28, 'I-IBAN': 29, 'B-ACCOUNTNUMBER': 30, 'I-ACCOUNTNUMBER': 31, 'B-BIC': 32, 'I-BIC': 33, 'B-IPV4': 34, 'I-IPV4': 35, 'B-STREETADDRESS': 36, 'I-STREETADDRESS': 37, 'B-CITY': 38, 'I-CITY': 39, 'B-ZIPCODE': 40, 'I-ZIPCODE': 41, 'B-USERNAME': 42, 'I-USERNAME': 43, 'B-IPV6': 44, 'I-IPV6': 45, 'B-CREDITCARDNUMBER': 46, 'I-CREDITCARDNUMBER': 47, 'B-VEHICLEVIN': 48, 'I-VEHICLEVIN': 49, 'B-SUFFIX': 50, 'I-SUFFIX': 51, 'B-AMOUNT': 52, 'I-AMOUNT': 53, 'B-CURRENCY': 54, 'I-CURR

In [12]:
# Define metrics calculation function
def compute_metrics(predictions, references):
    acc = accuracy_score(references, predictions)
    precision = precision_score(references, predictions, average="weighted", zero_division=1)
    recall = recall_score(references, predictions, average="weighted", zero_division=1)
    f1 = f1_score(references, predictions, average="weighted", zero_division=1)
    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [13]:
from transformers import DataCollatorForTokenClassification
from torch.utils.data import DataLoader
import numpy as np

def tokenize_and_align_labels(df, tokenizer, data_label_to_id, model_label_to_id):
    def normalize_label(label):
        # Remove suffixes like "_1", "_2"
        if "_" in label and label.split("_")[-1].isdigit():
            return "_".join(label.split("_")[:-1])
        return label

    def map_labels_to_model(data_label):
        normalized_label = normalize_label(data_label)  # Normalize dataset labels
        return model_label_to_id.get(normalized_label, model_label_to_id.get("O", -100))  # Default to "O"

    # Tokenize input text
    tokenized_inputs = tokenizer(
        list(df["unmasked_text"]),  # Convert Series to list of strings
        truncation=True,
        padding=True,
        max_length=512,
        return_tensors="pt",
        is_split_into_words=False,
    )

    labels = []
    for i, label_seq in enumerate(df["bio_labels"]):
        word_labels = eval(label_seq) if isinstance(label_seq, str) else label_seq  # Convert string back to list if needed
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens back to words
        token_labels = []
        for word_id in word_ids:
            if word_id is None:  # Special tokens
                token_labels.append(-100)
            elif word_id < len(word_labels):  # Ensure within bounds
                token_labels.append(map_labels_to_model(word_labels[word_id]))
            else:
                token_labels.append(-100)  # Ignore out-of-bounds tokens
        labels.append(token_labels)

    tokenized_inputs["labels"] = torch.tensor(labels)
    return tokenized_inputs

# Tokenize the training set
tokenized_train = tokenize_and_align_labels(eng_train, tokenizer, data_label_to_id, model_label_to_id)

# Create a DataLoader for the test set
train_dataset = torch.utils.data.TensorDataset(tokenized_train["input_ids"], tokenized_train["attention_mask"], tokenized_train["labels"])
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=False)

# Tokenize the test set
tokenized_test = tokenize_and_align_labels(eng_test, tokenizer, data_label_to_id, model_label_to_id)

# Create a DataLoader for the test set
test_dataset = torch.utils.data.TensorDataset(tokenized_test["input_ids"], tokenized_test["attention_mask"], tokenized_test["labels"])
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [14]:
for key, value in tokenized_train.items():
    print(f"{key}: {value.shape}")

for key, value in tokenized_test.items():
    print(f"{key}: {value.shape}")

input_ids: torch.Size([267, 140])
token_type_ids: torch.Size([267, 140])
attention_mask: torch.Size([267, 140])
labels: torch.Size([267, 140])
input_ids: torch.Size([68, 105])
token_type_ids: torch.Size([68, 105])
attention_mask: torch.Size([68, 105])
labels: torch.Size([68, 105])


In [15]:
id_to_label = {v: k for k, v in model_label_to_id.items()}

In [16]:
# # Find the optimized hyperparameters

# import optuna

# def objective(trial):
#     # Suggest hyperparameters
#     batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
#     temperature = trial.suggest_float("temperature", 0.5, 2.0)
#     use_fp16 = trial.suggest_categorical("use_fp16", [True, False])

#     # Configure inference settings
#     all_predictions, all_references = [], []
#     model.eval()

#     with torch.no_grad():
#         for batch in test_dataloader:
#             input_ids, attention_mask, labels = [t.to(device) for t in batch]

#             # Mixed precision inference
#             with torch.cuda.amp.autocast(enabled=use_fp16):
#                 outputs = model(input_ids, attention_mask=attention_mask)
#                 logits = outputs.logits / temperature  # Apply temperature scaling

#             # Get predictions
#             predictions = torch.argmax(logits, dim=-1).cpu().numpy()
#             labels = labels.cpu().numpy()

#             # Align predictions and references to only include non-special tokens
#             for pred, label in zip(predictions, labels):
#                 true_labels = [id_to_label[l] for l in label if l != -100]
#                 pred_labels = [
#                     id_to_label.get(p, "O") for p, l in zip(pred, label) if l != -100
#                 ]
#                 all_predictions.extend(pred_labels)
#                 all_references.extend(true_labels)

#     # Calculate metrics
#     metrics = compute_metrics(all_predictions, all_references)
#     return metrics["f1"]  # Optimize for F1 score

# # Hyperparameter optimization with Optuna
# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=20)

# # Display the best hyperparameters
# print("Best hyperparameters:", study.best_params)

In [ ]:
from torch.optim import AdamW
from transformers import get_scheduler
from torch.cuda.amp import autocast, GradScaler

# Initialize AMP scaler
scaler = GradScaler()

# Calculate the total number of training steps
num_epochs = 5
num_training_steps = len(train_dataloader) * num_epochs
num_warmup_steps = int(0.1 * num_training_steps)  # Use 10% of steps for warmup

# Define optimizer
learning_rate = 2e-5  # Start with a small learning rate
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Initialize the learning rate scheduler (optional)
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)

model.train()
step = 0
total_loss = 0

# Loop over epochs
for epoch in range(num_epochs):
    # Loop over batches in train_dataloader
    for batch in train_dataloader:
        input_ids, attention_mask, labels = [t.to(device) for t in batch]

        # Enable mixed precision for memory efficiency
        with autocast():
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

        # Backward pass
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()

        # Update the learning rate scheduler
        lr_scheduler.step()

        # Log progress every 10 steps
        if step % 10 == 0:  # Log every 10 steps
            avg_loss = total_loss / (step + 1)
            print(f"Epoch {epoch+1}/{num_epochs}, Step {step}/{num_training_steps}, Loss: {loss.item()}, Avg Loss: {avg_loss}")

        step += 1
        if step >= num_training_steps:  # Stop if maximum steps reached
            break
    if step >= num_training_steps:
        break

<ipython-input-17-19200e941cba>:6: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/usr/local/lib/python3.10/dist-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
<ipython-input-17-19200e941cba>:36: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


Epoch 1/5, Step 0/45, Loss: 4.757996559143066, Avg Loss: 4.757996559143066


In [ ]:
# Inference loop using the optimized hyperparameters
all_predictions, all_references = [], []
model.eval()

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [t.to(device) for t in batch]

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits / 0.9989592666538125  # Use optimized temperature

        # Get predictions
        predictions = torch.argmax(logits, dim=-1).cpu().numpy()
        labels = labels.cpu().numpy()

        # Align predictions and references to only include non-special tokens
        for pred, label in zip(predictions, labels):
            true_labels = [
                id_to_label[l] for l in label if l != -100
            ]  # Convert ground truth IDs to labels
            pred_labels = [
                id_to_label.get(p, "O") for p, l in zip(pred, label) if l != -100
            ]  # Convert predictions to labels, defaulting to "O" for unmapped IDs

            all_predictions.extend(pred_labels)
            all_references.extend(true_labels)

# Calculate metrics
metrics = compute_metrics(all_predictions, all_references)
print("Metrics:", metrics)

In [ ]:
print(f"all predictions: {all_predictions}")
print(f"number of all predictions: {len(all_predictions)}")
print(f"all references: {all_references}")
print(f"number of all references: {len(all_references)}")

In [ ]:
from sklearn.decomposition import PCA
import numpy as np

# Extract the unique classes from all_references and all_predictions
unique_labels = sorted(set(all_references) | set(all_predictions))
display_labels = [id_to_label[label] for label in unique_labels]

# Plot the confusion matrix with consistent labels
ConfusionMatrixDisplay.from_predictions(
    all_references,
    all_predictions,
    display_labels=display_labels
)
plt.title("Confusion Matrix (Filtered Labels)")
plt.show()

In [ ]:
from sklearn.metrics import classification_report
import pandas as pd

report = classification_report(all_references, all_predictions, output_dict=True)
df = pd.DataFrame(report).transpose()

# Plot F1 scores for each label
df["f1-score"].plot(kind="bar", figsize=(12, 6))
plt.title("F1 Score by Label")
plt.ylabel("F1 Score")
plt.xlabel("Labels")
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

import numpy as np

cm = confusion_matrix(all_references, all_predictions)
misclassified_indices = np.unravel_index(np.argsort(-cm, axis=None), cm.shape)
top_misclassifications = [
    (id_to_label[misclassified_indices[0][i]], id_to_label[misclassified_indices[1][i]], cm[misclassified_indices[0][i], misclassified_indices[1][i]])
    for i in range(10)  # Top 10 misclassifications
]

for true, predicted, count in top_misclassifications:
    print(f"True: {true}, Predicted: {predicted}, Count: {count}")

# Extract unique indices for the top misclassifications
unique_indices = list(set([misclassified_indices[0][i] for i in range(10)] +
                          [misclassified_indices[1][i] for i in range(10)]))

# Create a subset confusion matrix
subset_cm = cm[np.ix_(unique_indices, unique_indices)]
subset_labels = [list(id_to_label.values())[i] for i in unique_indices]

# Plot the subset confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=subset_cm, display_labels=subset_labels)
disp.plot(cmap="viridis", xticks_rotation="vertical", colorbar=True)
plt.title("Confusion Matrix For Top Misclassifications)")
plt.show()

In [ ]:
from sklearn.metrics import precision_recall_curve, PrecisionRecallDisplay

# Convert string labels to numeric IDs using id_to_label reverse mapping
label_to_id_reverse = {label: idx for idx, label in id_to_label.items()}

# Map predictions and references to numeric IDs
numeric_predictions = [label_to_id_reverse[label] for label in all_predictions]
numeric_references = [label_to_id_reverse[label] for label in all_references]

# Choose the label ID of interest (e.g., for binary classification, the positive class)
pos_label_id = label_to_id_reverse["B-CREDITCARDNUMBER"]  # Replace "B-YOUR_LABEL" with your target class

# Compute precision-recall curve
from sklearn.metrics import precision_recall_curve, PrecisionRecallDisplay

precision, recall, _ = precision_recall_curve(numeric_references, numeric_predictions, pos_label=pos_label_id)

# Plot the Precision-Recall curve
disp = PrecisionRecallDisplay(precision=precision, recall=recall)
disp.plot()
plt.title("Precision-Recall Curve")
plt.show()

# Flatten predictions and references for multiclass handling (if applicable)
# Note: This works best for binary or one-vs-all analysis.
# Example below assumes binary or flattened labels.
precision, recall, _ = precision_recall_curve(numeric_references, numeric_predictions, pos_label="B-CREDITCARDNUMBER")

# Display Precision-Recall curve
disp = PrecisionRecallDisplay(precision=precision, recall=recall)
disp.plot()
plt.title("Precision-Recall Curve")
plt.show()
